# YOLOv5를 사용한 Detection 수행 및 Detection 결과 저장
- Input: 웹캠의 스냅샷 or 업로드 이미지
- Output: Detection 된 Object의 bbox가 그려진 Image + Bbox 정보 Json
    - 체크박스를 만들어 Save dir에 Image와 Json을 저장할 수 있도록 함.
- 그 외 기능: 주어진 ID 및 비밀번호를 입력해서 로그인을 해야 Detector를 사용할 수 있도록 함.
    - ID: haejun
    - PWD: yang

In [ ]:
import gradio as gr
import torch, cv2, numpy as np, time, json
from pathlib import Path
from datetime import datetime

# ============== YOLOv5 로드 ==============
print("Loading YOLOv5 model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("ultralytics/yolov5", "yolov5s", trust_repo=True).to(device).eval()
if device == "cuda":
    model.half()  # 문제 있으면 주석 처리
names = getattr(model, "names", {})
print(f"Model loaded on {device}")

@torch.no_grad()
def detection(frame: np.ndarray, conf_threshold: float = 0.3):
    """RGB np.uint8 Image 1장 → (annotated RGB, status, raw_det_list)"""
    if frame is None:
        return None, "Status: please capture or upload an image.", []

    t0 = time.time()

    # Detection은 BGR 이미지로 진행
    bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    bgr = np.ascontiguousarray(bgr)

    results = model(bgr, size=640)
    det = results.xyxy[0].detach().float().cpu().numpy() if hasattr(results, "xyxy") else np.empty((0, 6))

    out = frame.copy()  # 출력은 RGB 유지
    people = 0
    kept = []  # JSON 저장용

    H, W = out.shape[:2]
    for *xyxy, conf, cls in det:
        conf = float(conf)
        cls  = int(cls)
        if conf < conf_threshold:
            continue

        x1, y1, x2, y2 = map(int, xyxy)
        w, h = max(0, x2 - x1), max(0, y2 - y1)
        cx, cy = x1 + w/2.0, y1 + h/2.0

        if cls == 0:  # person 카운트
            people += 1

        kept.append({
            "class_id": cls,
            "class_name": names.get(cls, str(cls)),
            "confidence": conf,
            "bbox_xyxy": [x1, y1, x2, y2],
            # "bbox_width":  w,
            # "bbox_height": h,
            # "bbox_area":   w * h,
            "center": [cx, cy]
        })

        # 시각화
        label = f"{names.get(cls, str(cls))} {conf:.2f}"
        cv2.rectangle(out, (x1, y1), (x2, y2), (0,255,0), 2)
        (tw, th), bl = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        y1t = max(0, y1 - th - bl - 3)
        cv2.rectangle(out, (x1, y1t), (x1+tw+6, y1t+th+bl+6), (0,120,0), -1)
        cv2.putText(out, label, (x1+3, y1t+th+3), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255,255,255), 1, cv2.LINE_AA)

    fps = 1.0 / max(1e-6, time.time() - t0)
    status = f"Status: Detected {people} people | FPS: {fps:.1f}"
    return out, status, kept

def detect_from_inputs(webcam_frame, uploaded_frame, conf, save_img, save_json, save_dir):
    """웹캠 우선, 없으면 업로드 Image 사용. 저장 옵션에 따라 파일 저장."""
    frame = webcam_frame if webcam_frame is not None else uploaded_frame
    out, status, kept = detection(frame, conf)

    # 저장 디렉토리 준비
    if (save_img or save_json) and out is not None:
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        stamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # 파일 베이스명
        base_jpg = f"{stamp}_det.jpg"
        base_json = f"{stamp}_det.json"

        if save_img:
            cv2.imwrite(str(Path(save_dir)/base_jpg), cv2.cvtColor(out, cv2.COLOR_RGB2BGR))
            status += f" | Saved image: {base_jpg}"

        if save_json:
            H, W = out.shape[:2]
            payload = {
                "source": base_jpg if save_img else None,
                "timestamp": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
                "image_size": [int(H), int(W)],
                "detections": kept
            }
            with open(Path(save_dir)/base_json, "w", encoding="utf-8") as f:
                json.dump(payload, f, ensure_ascii=False, indent=2)
            status += f" | Saved JSON: {base_json}"

    return out, status

# ============== Gradio UI ==============
with gr.Blocks() as demo:
    gr.Markdown("## YOLOv5 Detector + JSON Export\n"
                "- 웹캠 스냅샷(카메라 아이콘) 또는 Image 업로드 후 Run Detection.\n"
                "- 체크박스로 Image/JSON 저장을 선택할 수 있습니다.")

    with gr.Row():
        webcam_snap = gr.Image(label="Webcam Snapshot", sources=["webcam"], type="numpy")
        upload_img  = gr.Image(label="Or Upload an Image", type="numpy")
        result_img  = gr.Image(label="Detection Result", type="numpy")

    status_box = gr.Textbox(label="Processing Status", interactive=False)

    with gr.Row():
        conf      = gr.Slider(0.0, 1.0, value=0.3, step=0.05, label="Confidence Threshold")
        save_img  = gr.Checkbox(value=True,  label="Save annotated image")
        save_js   = gr.Checkbox(value=True,  label="Save bbox JSON")
        save_dir  = gr.Textbox(value="/workspace/out_snapshots", label="Save dir", scale=2)

    run_btn = gr.Button("Run Detection", variant="primary")

    run_btn.click(
        fn=detect_from_inputs,
        inputs=[webcam_snap, upload_img, conf, save_img, save_js, save_dir],
        outputs=[result_img, status_box]
    )

demo.queue().launch(server_port=8002, share=True, auth=("haejun", "yang"))

Loading YOLOv5 model...


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-10-15 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Model loaded on cuda
* Running on local URL:  http://127.0.0.1:8002
* Running on public URL: https://86d1f639dc30d82946.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am